In [ ]:
bucket_name = "banpu-interns-data-exchange"
model_dir = "models/test_model"
file_path = "models/test_model.tar.gz"

In [ ]:
import tarfile
import os

with tarfile.open(file_path, "w:gz") as tar:
    tar.add(model_dir, arcname=os.path.basename(model_dir))

In [ ]:
import boto3

# Use AWS CLI named profile
session = boto3.Session(profile_name="bivlab")

In [ ]:
# Upload model file to S3
s3 = session.client("s3")
s3.upload_file(file_path, bucket_name, file_path)

In [ ]:
import json

# Create SageMaker execution role
iam = session.client("iam")
role_name = "SageMakerRole"

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName=role_name,
        Description="Allows SageMaker to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
    )

    role_arn = create_role_response["Role"]["Arn"]
except iam.exceptions.EntityAlreadyExistsException:
    role_arn = iam.get_role(RoleName=role_name)["Role"]["Arn"]

# Attach AmazonSageMakerFullAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"

try:
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn=policy_arn
    )
except iam.exceptions.PolicyAlreadyExistsException:
    pass

# Attach AmazonS3FullAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"

try:
    iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
except iam.exceptions.PolicyAlreadyExistsException:
    pass

In [ ]:
import sagemaker
from sagemaker.session import Session

sagemaker_session = Session(boto_session=session)

In [ ]:
from sagemaker.tensorflow.serving import TensorFlowModel

# Deploy tensorflow model to Sagemaker endpoint
model_data = "s3://" + bucket_name + "/" + file_path

model = TensorFlowModel(
    role=role_arn,
    model_data=model_data,
    framework_version="2.14",
    sagemaker_session=sagemaker_session,
)

serverless_config = sagemaker.serverless.ServerlessInferenceConfig(
    memory_size_in_mb=4096,
    max_concurrency=10,
)

predictor = model.deploy(serverless_inference_config=serverless_config)
print(predictor.endpoint_name)